In [1]:
import numpy as np

In [2]:
from utils_graph_generation import load_graph, get_paths
from utils_graph_metrics import get_exp
from utils_graph_plot import plot_degree_distribution, plot_degree_probability_distribution, plot_degree_cummulative_distribution, plot_degree_complementary_cummulative_distribution

# Distribución de grados
Dado un grafo muestra tres gráficas:
* Un histograma con la frecuencia de los grados de los nodos normalizados según el número de nodos del grafo
* Un histograma con el acumulado de la frecuencia de los grados de los nodos normalizados según el número de nodos del grafo
* Un histograma con la inversa del acumulado de la frecuencia de los grados de los nodos normalizados según el número de nodos del grafo

In [3]:
# Dado un nombre de grafo, imprime las gráficas relativas a las distribuciones de grados de los nodos
def calc_plot_degree_distributions(name_graph, graphs_folder, plots_folder, G=None):
    if G is None:
        G = load_graph(name_graph, graphs_folder)

    # Normalizamos entre el número total de nodos
    points = np.sort(np.array(list(dict(G.degree()).values())).astype(float)/float(G.number_of_nodes()))

    get_exp([points], name_graph)

    plot_degree_distribution([points], name_graph, plots_folder)

    arr_deg_prob = plot_degree_probability_distribution([points], G.number_of_nodes(), name_graph, plots_folder)
    
    arr_deg_cum = plot_degree_cummulative_distribution(arr_deg_prob, name_graph, plots_folder)

    plot_degree_complementary_cummulative_distribution(arr_deg_cum, name_graph, plots_folder)


In [4]:
MODE = "hashtag"

MANIFESTACION = "nat"
hora = "429624"
#429674

#MANIFESTACION = "9n"
#hora = "437038"

# sale como max clust el 437040


graphs_folder, plots_folder = get_paths(MODE, MANIFESTACION, metric="degree")

calc_plot_degree_distributions(hora, graphs_folder, plots_folder)


Cargado el grafo de la hora 429624, numero de nodos: 642, numero de aristas: 4470
Calculating best minimal value for power law fit
alpha: 2.1930724612278993
x_min: 0.014018691588785047
L: 0.06040966884865273
D: 0.014018691588785047
parameters: None
-1.1556180984018436
0.33186398941958384


AttributeError: KS

In [ ]:
from utils_graph_metrics import tresh_normalization

In [ ]:
def calc_plot_degree_distributions_tres(name_graph, graphs_folder, plots_folder, G=None, arr_kt_plot=[5, 10, 20, 50, 100, 150, 200, 250]):
    
    # Cargamos el grafo inicial
    if G is None:
        G = load_graph(name_graph, graphs_folder)
    arr_points = []
    for kt in arr_kt_plot:
        F = tresh_normalization(G, kt)
        points_kt = np.sort(np.array(list(dict(F.degree()).values())).astype(float)/float(G.number_of_nodes()))
        arr_points.append(points_kt)

    get_exp(arr_points, arr_kt_plot)
    
    """plot_degree_distribution(arr_points, name_graph, plots_folder, arr_kt_plot=arr_kt_plot, alpha=0.45)

    arr_deg_prob = plot_degree_probability_distribution(arr_points, G.number_of_nodes(), name_graph, plots_folder, arr_kt_plot=arr_kt_plot)
    
    arr_deg_cum = plot_degree_cummulative_distribution(arr_deg_prob, name_graph, plots_folder, arr_kt_plot=arr_kt_plot)

    plot_degree_complementary_cummulative_distribution(arr_deg_cum, name_graph, plots_folder, arr_kt_plot=arr_kt_plot)"""

In [ ]:
graphs_folder, plots_folder = get_paths(MODE, MANIFESTACION, metric="degree")

calc_plot_degree_distributions_tres(hora, graphs_folder, plots_folder, arr_kt_plot=[0,5,10,15,20,25,30,35,40,45,50])

Cargado el grafo de la hora 429624, numero de nodos: 642, numero de aristas: 4470
[0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763 0.00155763
 0.00155763 0.00155763 0.00155763 0.00155763 0.